# AI in Capacity Management Demo

Team - Siddhant Sahu

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from search import parse_demand, get_weight
from retrieval import Retrieval

<IPython.core.display.Javascript object>

### Input 1 - Demand Data

Demand data stored in a csv file.

In [3]:
demand_data = pd.read_csv("../data/demand.csv", sep="\t").fillna("")
demand_data

,Requestor,Requestor Service Line,Requestor Sub ServiceLine,Requestor SMU,Job Title,Rank,No of Resources_required,Country,Location,Alternate Location,Technical Skill 1,Technical Skill 2,Technical Skill 3,Functional Skill 1,Functional Skill 2,Functional Skill 3,Process Skill 1,Process Skill 2,Process Skill 3,Min Experience
0,Req_1,ServiceLine3,SubserviceLine4,SMU2,Financial Risk Analyst,Rank_3,1,India,Bangalore,,Microsoft Excel,Microsoft Word,,Risk Analysis,Analysis,Accounting,Communication,Documentation,Team Skill,8
1,Req_2,ServiceLine1,SubserviceLine1,SMU1,Data Visualisation Engineer,Rank_5,1,India,Kochi,,Dashboarding,Power BI,SQL,Requirement Analysis,,,Communication,Problem Solving,Team Skill,1
2,Req_3,ServiceLine2,SubserviceLine2,SMU3,ML Engineer,Rank_3,1,India,Bangalore,,Artificial Intelligence,ML,Python,Business Analysis,Requirement Analysis,Data Modelling,Problem Solving,Analytical Ability,Communication,8
3,Req_4,ServiceLine3,SubserviceLine1,SMU3,Account Reporting Specialist,Rank_4,2,Poland,Warsaw,,Microsoft Excel,VBA,,Accounting,Internal Audit,Reporting,Communication,Problem Solving,Analytical Ability,5
4,Req_5,ServiceLine3,SubserviceLine4,SMU2,Business Analyst,Rank_4,1,Germany,Koln,,Microsoft Visio,Microsoft Office,,Business Analysis,Taxation,Business Case Modelling,Problem Solving,Communication,Documentation,5
5,Req_6,ServiceLine2,SubserviceLine3,SMU1,Technical Lead - Web Technologies,Rank_3,1,India,Kochi,,Java,HTML5,Cloud,Requirement Analysis,Business Analysis,,Problem Solving,Analytical Ability,Communication,8
6,Req_7,ServiceLine2,SubserviceLine4,SMU2,Web Developer,Rank_5,1,India,Bangalore,,HTML5,React,Azure,Requirement Analysis,,,Communication,Team Skill,Analytical Ability,1


<IPython.core.display.Javascript object>

### Input 2 - Weights for Search Factors

Weight factors also stored in a csv file.

In [4]:
weights_data = pd.read_csv("../data/weights.csv", sep="\t")
weights_data

,criteria,technical,functional,process,experience,rank,location,bench_age
0,ServiceLine1,0.10,0.30,0.10,0.1,0.1,0.3,0.0
1,ServiceLine2,0.10,0.25,0.05,0.1,0.0,0.4,0.1
2,ServiceLine3,0.15,0.20,0.05,0.2,0.0,0.1,0.3


<IPython.core.display.Javascript object>

### Interactive Demo

Check results when different combinations of demand + weight data are selected.

In [5]:
@interact
def show_search_results(
    requestor=[f"Req_{i}" for i in range(1, 8)],
    criteria=[f"ServiceLine{i}" for i in range(1, 4)],
    sort_by_dept=False,
):
    obj = Retrieval()
    demand_idx = int(requestor.replace("Req_", "")) - 1
    weight_idx = int(criteria.replace("ServiceLine", "")) - 1
    demand = parse_demand("../data/demand.csv", demand_idx)
    weight = get_weight("../data/weights.csv", weight_idx)
    res = obj.get_results(demand, weight, sort_by_dept).rename(
        columns={"years_of_experience": "experience"}
    )
    cm = sns.light_palette("green", as_cmap=True)
    return (
        res.style.background_gradient(cmap=cm, subset=["fitment"])
        .format(
            {
                "fitment": "{:.1%}",
                "technical": "{:.0f}",
                "functional": "{:.0f}",
                "process": "{:.0f}",
            }
        )
        .hide_index()
        .hide_columns("dept_similarity")
    )

interactive(children=(Dropdown(description='requestor', options=('Req_1', 'Req_2', 'Req_3', 'Req_4', 'Req_5', …

<IPython.core.display.Javascript object>

### Explanation

* When `sort_by_dept` is checked, the results will be sorted by department first, fitment scores next.
* When unchecked, the results will be sorted by fitment scores, without considering department of the employee.